<a href="https://colab.research.google.com/github/archanashokeeniitg/machineLearning/blob/main/ML_Ass_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

FA21: CMPE-257 Sec 48 - Machine Learning

Archana Shokeen (015237378)

Assignment-Week-8

San Jose State University


#**Veracity Factor - “Stance Detection”**

##**Description**


Fake news includes false stories about all sorts of topics that are crafted in order to appear real and get traffic and social shares on Facebook and Twitter. Stance Detection is one of the factors influencing fake news detection. Stance detection on social media is an emerging opinion mining paradigm for various social and political applications in which sentiment analysis may be sub-optimal.  For humans, its impractical to check the fact's every piece of information produced by the media. To overcome this we can make use of natural language processing (NLP) techniques to automate stance detection to determine the quality of the news source. It considers what other organizations write about the same headline. A body of the text is claimed to agree, disagree, discuss, or be unrelated to a headline.

##**Academic Paper** - ”Stance detection on social media: State of the art and trends”

**Reference**

ALDayel, A., & Magdy, W. (2021). Stance detection on social media: State of the art and trends. Information Processing & Management, 58(4), 102597. https://doi.org/10.1016/j.ipm.2021.102597

##**MicroFactors**

* Agree
* Disagree
* Unrelated
* Discuss

## **DataScraping and Data source**
The website used "https://www.politifact.com/" using Beautiful soup





#Data Scraping from Poloitofact



In [2]:
#Import the dependencies
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time

In [3]:
#Create lists to store the scraped data
authors = []
dates = []
statements = []
sources = []
targets = []

In [4]:
#Create a function to scrape the site
def scrape_website(page_number):
  page_num = str(page_number) #Convert the page number to a string
  URL = 'https://www.politifact.com/factchecks/list/?page='+page_num #append the page number to complete the URL
  webpage = requests.get(URL)  #Make a request to the website
  #time.sleep(3)
  soup = BeautifulSoup(webpage.text, "html.parser") #Parse the text from the website
  #Get the tags and it's class
  statement_footer =  soup.find_all('footer',attrs={'class':'m-statement__footer'})  #Get the tag and it's class
  statement_quote = soup.find_all('div', attrs={'class':'m-statement__quote'}) #Get the tag and it's class
  statement_meta = soup.find_all('div', attrs={'class':'m-statement__meta'})#Get the tag and it's class
  target = soup.find_all('div', attrs={'class':'m-statement__meter'}) #Get the tag and it's class
  #loop through the footer class m-statement__footer to get the date and author
  for i in statement_footer:
    link1 = i.text.strip()
    name_and_date = link1.split()
    first_name = name_and_date[1]
    last_name = name_and_date[2]
    full_name = first_name+' '+last_name
    month = name_and_date[4]
    day = name_and_date[5]
    year = name_and_date[6]
    date = month+' '+day+' '+year
    dates.append(date)
    authors.append(full_name)
  #Loop through the div m-statement__quote to get the link
  for i in statement_quote:
    link2 = i.find_all('a')
    statements.append(link2[0].text.strip())
 #Loop through the div m-statement__meta to get the source
  for i in statement_meta:
    link3 = i.find_all('a') #Source
    source_text = link3[0].text.strip()
    sources.append(source_text)
  #Loop through the target or the div m-statement__meter to get the facts about the statement (True or False)
  for i in target:
    fact = i.find('div', attrs={'class':'c-image'}).find('img').get('alt')
    targets.append(fact)


In [5]:
#Loop through 'n-1' webpages to scrape the data
n=5
for i in range(1, n):
  scrape_website(i)

In [6]:
#Create a new dataFrame 

data = pd.DataFrame(columns = ['author', 'date']) 
data.apply(lambda col: col.drop_duplicates().reset_index(drop=True))
data['author'] = authors
# data['statement'] = statements
# data['source'] = sources
data['date'] = dates
# data['target'] = targets
#Show the data set
data

,author,date
0,Kiannah Sepeda-Miller,"October 31, 2021"
1,Tom Kertscher,"October 29, 2021"
2,Monique Curet,"October 29, 2021"
3,Samantha Putterman,"October 29, 2021"
4,Ciara O'Rourke,"October 29, 2021"
...,...,...
115,Samantha Putterman,"October 11, 2021"
116,Bill McCarthy,"October 11, 2021"
117,Gabrielle Settles,"October 11, 2021"
118,Jon Greenberg,"October 11, 2021"


In [7]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd

pagesToGet= 1

upperframe=[]  
for page in range(1,pagesToGet+1):
    print('processing page :', page)
    url = 'https://www.politifact.com/factchecks/list/?page='+str(page)
    print(url)
    
    #an exception might be thrown, so the code should be in a try-except block
    try:
        #use the browser to get the url. This is suspicious command that might blow up.
        page=requests.get(url)                             # this might throw an exception if something goes wrong.
    
    except Exception as e:                                   # this describes what to do if an exception is thrown
        error_type, error_obj, error_info = sys.exc_info()      # get the exception information
        print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
        print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
        continue                                              #ignore this page. Abandon this and go back.
    time.sleep(2)   
    soup=BeautifulSoup(page.text,'html.parser')
    frame=[]
    links=soup.find_all('li',attrs={'class':'o-listicle__item'})
    print(len(links))
    filename="NEWS.csv"
    f=open(filename,"w", encoding = 'utf-8')
    headers="Statement,Link,Date, Source, Label\n"
    f.write(headers)
    
    for j in links:
        Statement = j.find("div",attrs={'class':'m-statement__quote'}).text.strip()
        Link = "https://www.politifact.com"
        Link += j.find("div",attrs={'class':'m-statement__quote'}).find('a')['href'].strip()
        Date = j.find('div',attrs={'class':'m-statement__body'}).find('footer').text[-14:-1].strip()
        Source = j.find('div', attrs={'class':'m-statement__meta'}).find('a').text.strip()
        Label = j.find('div', attrs ={'class':'m-statement__content'}).find('img',attrs={'class':'c-image__original'}).get('alt').strip()
        frame.append((Statement,Link,Date,Source,Label))
        f.write(Statement.replace(",","^")+","+Link+","+Date.replace(",","^")+","+Source.replace(",","^")+","+Label.replace(",","^")+"\n")
    upperframe.extend(frame)
f.close()
data=pd.DataFrame(upperframe, columns=['Statement','Link','Date','Source','Label'])
data.head()

processing page : 1
https://www.politifact.com/factchecks/list/?page=1
30


,Statement,Link,Date,Source,Label
0,"The migrant caravan from Tapachula, Mexico “is...",https://www.politifact.com/factchecks/2021/oct...,"ober 31, 2021",Mary Miller,pants-fire
1,Blood “starts to clot in a minute or two” afte...,https://www.politifact.com/factchecks/2021/oct...,"ober 29, 2021",Instagram posts,false
2,“Japan drops vax rollout”,https://www.politifact.com/factchecks/2021/oct...,"ober 29, 2021",Hal Turner Radio Show,pants-fire
3,Virginia voters are being turned away from pol...,https://www.politifact.com/factchecks/2021/oct...,"ober 29, 2021",Facebook posts,barely-true
4,In-N-Out cups say “F--- Joe Biden.”,https://www.politifact.com/factchecks/2021/oct...,"ober 29, 2021",Viral image,false


In [8]:
data.to_csv('politifact_scraped_data.csv')